In [6]:
# !pip install dask[dataframe]

In [7]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error


In [8]:
model_data = pd.read_csv('file/ml_data.csv')
dong_st = pd.read_csv('file/merged_result.csv')

In [9]:
model_data

,년분기,행정동_코드,행정동,업종명,월매출,주중매출,주말매출,인구수,점포수,지역생활인구,장기외국인,단기외국인,주차장면적(면),주차장개수(개소),학교수,학생수,버스정류장수
0,20191,11710631,가락1동,미곡판매,19578367610,17153991613,2424375997,20186,7,523724,13892,1772,20942,83,3,1586,14
1,20191,11710631,가락1동,분식전문점,489000000,314598272,174401728,20186,9,523724,13892,1772,20942,83,3,1586,14
2,20191,11710631,가락1동,편의점,3225958567,2517887516,708071051,20186,4,523724,13892,1772,20942,83,3,1586,14
3,20191,11710631,가락1동,한식음식점,13351806795,8920140911,4431665884,20186,74,523724,13892,1772,20942,83,3,1586,14
4,20191,11710631,가락1동,의약품,53487294,48704751,4782543,20186,7,523724,13892,1772,20942,83,3,1586,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374576,20242,11590605,흑석동,화장품,495453596,358209024,137244572,32491,15,848734,77201,6202,16937,298,0,0,59
374577,20242,11590605,흑석동,화초,155767472,115723892,40043580,32491,15,848734,77201,6202,16937,298,0,0,59
374578,20242,11590605,흑석동,가구,169511052,169511052,0,32491,4,848734,77201,6202,16937,298,0,0,59
374579,20242,11590605,흑석동,철물점,19005645,16520754,2484891,32491,4,848734,77201,6202,16937,298,0,0,59


In [10]:
print(model_data.columns)

Index(['년분기', '행정동_코드', '행정동', '업종명', '월매출', '주중매출', '주말매출', '인구수', '점포수',
       '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수',
       '버스정류장수'],
      dtype='object')


In [11]:
columns_to_drop = ['월매출','주말매출','행정동_코드']
filtered_data = model_data.drop(columns=columns_to_drop)

print(filtered_data.head())

     년분기   행정동    업종명         주중매출    인구수  점포수  지역생활인구  장기외국인  단기외국인  \
0  20191  가락1동   미곡판매  17153991613  20186    7  523724  13892   1772   
1  20191  가락1동  분식전문점    314598272  20186    9  523724  13892   1772   
2  20191  가락1동    편의점   2517887516  20186    4  523724  13892   1772   
3  20191  가락1동  한식음식점   8920140911  20186   74  523724  13892   1772   
4  20191  가락1동    의약품     48704751  20186    7  523724  13892   1772   

   주차장면적(면)  주차장개수(개소)  학교수   학생수  버스정류장수  
0     20942         83    3  1586      14  
1     20942         83    3  1586      14  
2     20942         83    3  1586      14  
3     20942         83    3  1586      14  
4     20942         83    3  1586      14  


In [12]:
filtered_data.columns

Index(['년분기', '행정동', '업종명', '주중매출', '인구수', '점포수', '지역생활인구', '장기외국인', '단기외국인',
       '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'],
      dtype='object')

In [13]:
# 모델 학습에 필요한 컬럼만 선택 (주말매출, 업종명, 행정동 등)
X = model_data[['업종명', '행정동', '인구수', '점포수', '지역생활인구', '장기외국인', 
                '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수']]

# 주말매출을 타겟 변수로 설정
y = model_data['주말매출']

In [14]:
# One-Hot Encoding 적용 (업종명, 행정동)
X = pd.get_dummies(X, columns=['업종명', '행정동'], drop_first=True)

In [15]:
# # 범주형 변수를 Label Encoding으로 변환 (One-Hot Encoding 대신)
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# X['업종명'] = le.fit_transform(X['업종명'])
# X['행정동'] = le.fit_transform(X['행정동'])


In [16]:
import dask.dataframe as dd

# Dask DataFrame으로 읽기
dask_df = dd.from_pandas(model_data, npartitions=4)

# Dask DataFrame에서 작업 수행
X_dask = dask_df[['업종명', '행정동', '인구수', '점포수', '지역생활인구', '장기외국인',
                  '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수']]
y_dask = dask_df['주말매출']


In [17]:
# 데이터의 10%만 샘플링해서 사용
model_data_sample = model_data.sample(frac=0.1, random_state=42)

# 다시 X와 y로 분할
X_sample = model_data_sample[['업종명', '행정동', '인구수', '점포수', '지역생활인구', '장기외국인',
                              '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수']]
y_sample = model_data_sample['주말매출']


In [18]:
# 데이터 타입 최적화 (예: int64 -> int32, float64 -> float32)
X = X.astype({'인구수': 'int32', '점포수': 'int32', '지역생활인구': 'float32', '장기외국인': 'float32'})

In [19]:
# 데이터셋 크기 확인
print(X.memory_usage(deep=True))

Index           132
인구수         1498324
점포수         1498324
지역생활인구      1498324
장기외국인       1498324
             ...   
행정동_효창동      374581
행정동_후암동      374581
행정동_휘경1동     374581
행정동_휘경2동     374581
행정동_흑석동      374581
Length: 495, dtype: int64


특징변수(X)와 타겟변수(y) 준비

In [20]:
X = model_data[['업종명', '행정동', '인구수', '점포수', '지역생활인구', '장기외국인',
                '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수']]
y = model_data['주말매출']

범주형 변수 인코딩

In [21]:
# '업종명'과 '행정동'을 범주형 변수로 변환
X['업종명'] = X['업종명'].astype('category')
X['행정동'] = X['행정동'].astype('category')

# 범주형 변수를 숫자형으로 변환
X['업종명'] = X['업종명'].cat.codes
X['행정동'] = X['행정동'].cat.codes

C:\Users\human\AppData\Local\Temp\ipykernel_13548\2535871467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['업종명'] = X['업종명'].astype('category')
C:\Users\human\AppData\Local\Temp\ipykernel_13548\2535871467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['행정동'] = X['행정동'].astype('category')
C:\Users\human\AppData\Local\Temp\ipykernel_13548\2535871467.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

훈련 데이터와 테스트 데이터로 분할

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

모델 스케일링

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# 1. 스케일러 객체 생성
scaler = StandardScaler()

# 2. 훈련 데이터(X_train) 스케일링
X_train_scaled = scaler.fit_transform(X_train)

# 3. 테스트 데이터(X_test) 스케일링 (훈련 데이터로부터 학습한 스케일러를 사용)
X_test_scaled = scaler.transform(X_test)

모델 훈련 및 예측

In [24]:
# 모델 생성
model = LinearRegression()

# 스케일링된 훈련 데이터로 모델 학습
model.fit(X_train_scaled, y_train)

# 예측 (테스트 데이터로 예측)
y_pred = model.predict(X_test_scaled)

# 예측 결과 확인
print(y_pred)

[2.07302289e+08 2.34973712e+08 1.19391223e+08 ... 2.89933869e+08
 2.39952383e+08 1.15670419e+08]


모델 평가 

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# MAE, MSE, R² 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 평가 지표 출력
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R²): {r2}')

# 결과 
# Mean Absolute Error (MAE): 358628121.2588812
# Mean Squared Error (MSE): 1.2215718144740352e+19
# R-squared (R²): 0.07450361799097005

Mean Absolute Error (MAE): 358628121.2588812
Mean Squared Error (MSE): 1.2215718144740352e+19
R-squared (R²): 0.07450361799097005


In [26]:
# 모든데이터 + 지하철 데이터 병합 

# 데이터 확인
dong_st

# NaN 값 0으로 채우기 
dong_st = dong_st.fillna(0)
dong_st

,행정동,년분기,지하철역_갯수,총승하차인원_합계,지하철역_목록,시군구,총_승객수
0,가락1동,20191,2,0.0,"송파, 가락시장",0.0,0.0
1,가락1동,20192,2,0.0,"송파, 가락시장",0.0,0.0
2,가락1동,20193,2,0.0,"가락시장, 송파",0.0,0.0
3,가락1동,20194,2,0.0,"가락시장, 송파",0.0,0.0
4,가락1동,20201,2,0.0,"송파, 가락시장",0.0,0.0
...,...,...,...,...,...,...,...
2786,흑석동,20204,1,0.0,흑석(중앙대입구),0.0,0.0
2787,흑석동,20211,1,1103017.0,흑석(중앙대입구),0.0,0.0
2788,흑석동,20212,1,1245076.0,흑석(중앙대입구),0.0,0.0
2789,흑석동,20213,1,1101042.0,흑석(중앙대입구),0.0,0.0


In [27]:
model_data.columns

Index(['년분기', '행정동_코드', '행정동', '업종명', '월매출', '주중매출', '주말매출', '인구수', '점포수',
       '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수',
       '버스정류장수'],
      dtype='object')

In [28]:
import pandas as pd 
merged_df_sw = pd.merge(model_data, dong_st, on=['행정동', '년분기'], how='inner')
print(merged_df_sw)

          년분기    행정동_코드   행정동    업종명          월매출         주중매출        주말매출  \
0       20191  11710631  가락1동   미곡판매  19578367610  17153991613  2424375997   
1       20191  11710631  가락1동  분식전문점    489000000    314598272   174401728   
2       20191  11710631  가락1동    편의점   3225958567   2517887516   708071051   
3       20191  11710631  가락1동  한식음식점  13351806795   8920140911  4431665884   
4       20191  11710631  가락1동    의약품     53487294     48704751     4782543   
...       ...       ...   ...    ...          ...          ...         ...   
104942  20214  11590605   흑석동   의료기기    266753210    238570083    28183127   
104943  20214  11590605   흑석동     가구    501098460    178432686   322665774   
104944  20214  11590605   흑석동  커피-음료   1216604730    924008877   292595853   
104945  20214  11590605   흑석동    네일숍      9142784      7826264     1316520   
104946  20214  11590605   흑석동  분식전문점   1132851636    883201809   249649827   

          인구수  점포수  지역생활인구  ...  주차장면적(면)  주차장개수(개소)  학교수   학생수

In [29]:
print(merged_df_sw.columns)

Index(['년분기', '행정동_코드', '행정동', '업종명', '월매출', '주중매출', '주말매출', '인구수', '점포수',
       '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수',
       '버스정류장수', '지하철역_갯수', '총승하차인원_합계', '지하철역_목록', '시군구', '총_승객수'],
      dtype='object')


범주형 변수 one-hot-encoding

In [30]:
# One-Hot Encoding 적용
merged_df_sw = pd.get_dummies(merged_df_sw, columns=['업종명', '행정동'], drop_first=True)

In [31]:
merged_df_sw = pd.get_dummies(merged_df_sw, columns=['지하철역_목록'], drop_first=True)

X, y 선정

In [32]:
X = merged_df_sw.drop(['행정동_코드', '월매출','주중매출','주말매출'], axis=1)
y = merged_df_sw['주말매출'] # '주말매출'을 목표 변수로 설정 

훈련데이터와 테스트데이터로 분할 

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

데이터 스케일링(표준화)

In [34]:
scaler = StandardScaler() 
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

회귀 모델 학습

In [35]:
model = LinearRegression() 
model.fit(X_train_scaled, y_train)

LinearRegression()

예측

In [36]:
y_pred = model.predict(X_test_scaled)

성능 평가 

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score 
mae2 = mean_absolute_error(y_test, y_pred)
mse2 = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f"Mean Squared Error (MSE): {mse}")
print(f'R-squared (R²): {r2}')

# 결과 
# Mean Absolute Error (MAE): 358628121.2588812
# Mean Squared Error (MSE): 1.2215718144740352e+19
# R-squared (R²): 0.0804555037225968

Mean Absolute Error (MAE): 358628121.2588812
Mean Squared Error (MSE): 1.2215718144740352e+19
R-squared (R²): 0.0804555037225968


In [ ]:
# 처음 모델 성능
# Mean Absolute Error (MAE): 358628121.2588812
# Mean Squared Error (MSE): 1.2215718144740352e+19
# R-squared (R²): 0.07450361799097005

랜덤 포레스트

In [ ]:
# 랜덤 포레스트 회귀 모델 임포트
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 랜덤 포레스트 모델 정의
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# 모델 학습
rf_model.fit(X_train_scaled, y_train)

# 예측
y_pred_rf = rf_model.predict(X_test_scaled)

# 성능 평가
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# 성능 지표 출력
print(f'Mean Absolute Error (MAE): {mae_rf}')
print(f'Mean Squared Error (MSE): {mse_rf}')
print(f'R-squared (R²): {r2_rf}')

# 결과 
# Mean Absolute Error (MAE): 94421552.73414546
# Mean Squared Error (MSE): 1.4753923424747863e+18
# R-squared (R²): 0.9151077629925737

Mean Absolute Error (MAE): 94421552.73414546
Mean Squared Error (MSE): 1.4753923424747863e+18
R-squared (R²): 0.9151077629925737


하이퍼파라미터 튜닝 (그리드서치)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],  # 트리의 개수
    'max_depth': [None, 10, 20, 30],   # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],   # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],     # 리프 노드에 있어야 할 최소 샘플 수
    'max_features': ['auto', 'sqrt'],  # 분할에 사용할 특성 수
}

# 랜덤 포레스트 모델 생성
rf_model = RandomForestRegressor(random_state=42)

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

# 최적 파라미터 출력
print(f'Best Parameters: {grid_search.best_params_}')

# 최적 모델을 사용하여 예측
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)

# 성능 평가
mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f'Mean Absolute Error (MAE) after tuning: {mae_best}')
print(f'Mean Squared Error (MSE) after tuning: {mse_best}')
print(f'R-squared (R²) after tuning: {r2_best}')

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


In [39]:
# 모델 성능 개선 방법 
# 상관 분석 
# 랜덤 포레스트

전체 특성 간의 상관 관계 분석

In [47]:
# 결측값을 0으로 채우기
merged_df_sw = merged_df_sw.fillna(0)

# 상관 행렬 계산
correlation_matrix = merged_df_sw.corr()

# 상관 행렬 출력
print(correlation_matrix)

                                 년분기    행정동_코드       월매출      주중매출      주말매출  \
년분기                     1.000000e+00  0.013197  0.002696  0.003487  0.000401   
행정동_코드                  1.319743e-02  1.000000  0.007984  0.007166  0.008581   
월매출                     2.695741e-03  0.007984  1.000000  0.979644  0.882923   
주중매출                    3.486860e-03  0.007166  0.979644  1.000000  0.770698   
주말매출                    4.014454e-04  0.008581  0.882923  0.770698  1.000000   
...                              ...       ...       ...       ...       ...   
지하철역_목록_화랑대(서울여대입구)    -5.336660e-04 -0.033040 -0.005509 -0.005627 -0.004325   
지하철역_목록_회기              3.694526e-07 -0.073087 -0.006705 -0.007071 -0.004745   
지하철역_목록_회현(남대문시장), 서울역  3.627975e-02 -0.096319  0.007615  0.009933  0.000940   
지하철역_목록_효창공원앞           9.030672e-04 -0.084959 -0.005836 -0.006007 -0.004475   
지하철역_목록_흑석(중앙대입구)      -1.527909e-04  0.054809 -0.003179 -0.002885 -0.003342   

                             인구수       

In [ ]:
# 필요한 변수만 선택
selected_features = ['지하철역_갯수', '총승하차인원_합계', '주말매출']

# 상관행렬 계산
correlation_matrix = merged_df_sw[selected_features].corr()

# 상관행렬 출력
print(correlation_matrix)

# 상관행렬 시각화 (heatmap)
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True)
plt.show()